In [1]:
import pandas as pd
import numpy as np
import tiktoken
from pypdf import PdfReader
import os
import pickle


import openai

#openai.api_key = 

In [1]:
pip install streamlit-extras

  Using cached streamlit_extras-0.3.5-py3-none-any.whl (66 kB)
  Using cached streamlit_faker-0.0.3-py3-none-any.whl (14 kB)
  Using cached streamlit_camera_input_live-0.2.0-py3-none-any.whl (6.6 kB)
  Using cached streamlit_vertical_slider-1.0.2-py3-none-any.whl (624 kB)
  Using cached st_annotated_text-4.0.1-py3-none-any.whl (9.0 kB)
  Using cached streamlit_image_coordinates-0.1.6-py3-none-any.whl (6.3 kB)
  Using cached streamlit_card-0.0.61-py3-none-any.whl (680 kB)
  Using cached streamlit_embedcode-0.1.2-py3-none-any.whl (3.5 kB)
  Using cached htbuilder-0.6.2-py3-none-any.whl (12 kB)
  Using cached streamlit_keyup-0.2.0-py3-none-any.whl (7.4 kB)
  Using cached markdownlit-0.0.7-py3-none-any.whl (15 kB)
  Using cached streamlit_toggle_switch-1.0.2-py3-none-any.whl (635 kB)
  Using cached more_itertools-10.1.0-py3-none-any.whl (55 kB)
  Using cached favicon-0.7.0-py2.py3-none-any.whl (5.9 kB)
  Using cached pymdown_extensions-10.3.1-py3-none-any.whl (241 kB)
  Using cached Faker-


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\misc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# ########### NRGI ERFABLADE ##############
# df = pd.DataFrame(columns=['Kilde','content', 'tokens'])

# reader = PdfReader("Docs/Skadesvurdering_appen.pdf")

# i = 0
# for page in reader.pages:
#     df.loc[i, 'Kilde'] = 'NRGI ERFAblade side ' + str(i+1)
#     df.loc[i, 'content'] = page.extract_text()
#     i+=1
    
# df.head(5)

In [3]:
# ########### Håndbog ##############
# reader = PdfReader("Docs/haandbog_for_beskikkede_bygningssagkyndige_final_rev_apr_2021.pdf")

# i = df.shape[0]
# j=0
# for page in reader.pages:
#     df.loc[i, 'Kilde'] = 'Håndbog for beskikkede bygningssagkyndige side ' + str(j+1)
#     df.loc[i, 'content'] = page.extract_text()
#     i+=1
#     j+=1

In [4]:
# ########### Sikkerhedsstyrelsen guider ##############

# directory = 'Docs/Sikkerhedsstyrelsen guider/'

# for filename in os.listdir(directory):
#     #print(os.path.join(directory, filename))
#     reader = PdfReader(os.path.join(directory, filename))

#     i = df.shape[0]
#     for page in reader.pages:
#         df.loc[i, 'Kilde'] = 'Sikkerhedsstyrelsen guider: ' + str(filename)
#         df.loc[i, 'content'] = page.extract_text()
#         i+=1

In [5]:
# df.shape[0]

In [6]:
# enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

# for i in range(df.shape[0]):
#     df.loc[i, 'tokens'] = len(enc.encode(df.loc[i, 'content']))
#     i+=1

# df.to_csv('df_enc.csv')
#df = df.set_index(["Kilde"])


In [7]:
df = pd.read_csv('df_enc.csv')
df.tail(5)

,Unnamed: 0,Kilde,content,tokens
101,101,Sikkerhedsstyrelsen guider: guide_23_-_tagpapt...,\n \n \nGuide til vurdering af skader \n \n ...,1024
102,102,Sikkerhedsstyrelsen guider: guide_26_-_udvendi...,\n \n \nGuide til vurdering af skader \n \n ...,1468
103,103,Sikkerhedsstyrelsen guider: he-guide_33_-_vaad...,\n \n \nGuide til vurdering af skader \n \n ...,1114
104,104,Sikkerhedsstyrelsen guider: huseftersyn_guide_...,\n \n \nGuide til vurdering af skader \n \n ...,880
105,105,Sikkerhedsstyrelsen guider: huseftersyn_guide_...,\n \n \nGuide til vurdering af skader \n \n ...,1301


In [8]:
EMBEDDING_MODEL = "text-embedding-ada-002"
COMPLETIONS_MODEL = "gpt-3.5-turbo"

In [9]:
## This code was written by OpenAI: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }



In [10]:
# document_embeddings = compute_doc_embeddings(df)

# # An example embedding:
# example_entry = list(document_embeddings.items())[0]
# print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

In [11]:
# # save dictionary to person_data.pkl file
# with open('document_embeddings.pkl', 'wb') as fp:
#     pickle.dump(document_embeddings, fp)
#     print('dictionary saved successfully to file')

In [12]:
document_embeddings = pd.read_pickle(r'document_embeddings.pkl')

In [13]:
df.to_csv('df.csv', index=False, header=True)

In [14]:
## This code was written by OpenAI: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb


def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_by_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [15]:
order_by_similarity("Can the commission implement acts for exchanging information?", document_embeddings)[:5]

[(0.7523993440582353, 65),
 (0.7488279736787914, 61),
 (0.7475312944100329, 56),
 (0.7474719702964934, 64),
 (0.7465778252624518, 59)]

In [16]:
order_by_similarity("Do I have permission to review my information?", document_embeddings)[:5]

[(0.7388670219647755, 83),
 (0.7377875645925425, 69),
 (0.7376447071316635, 61),
 (0.7351218741454887, 54),
 (0.7332231007471646, 82)]

In [17]:
MAX_SECTION_LEN = 2000
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [18]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_by_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
        
    return chosen_sections, chosen_sections_len

In [19]:
def answer_with_gpt_4(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    messages = [
        {"role" : "system", "content":"Du er en rådgiver chatbot der kun kan svare ud fra den kontekst du er blevet tilført her. Hvis du ikke kan svare på spørgsmålet skal du svare 'Svaret er ikke i ERFA bladene, håndbogen eller Sikkerhedsstyrelsens guider.'"}
    ]
    prompt, section_lenght = construct_prompt(
        query,
        document_embeddings,
        df
    )
    if show_prompt:
        print(prompt)

    context= ""
    for article in prompt:
        context = context + article 

    context = context + '\n\n --- \n\n + ' + query

    messages.append({"role" : "user", "content":context})
    response = openai.ChatCompletion.create(
        model=COMPLETIONS_MODEL,
        temperature=0.0,
        max_tokens=2000,
        messages=messages
        )

    return '\n' + response['choices'][0]['message']['content'], section_lenght

In [20]:
df.tail(50)

,Unnamed: 0,Kilde,content,tokens
56,56,Håndbog for beskikkede bygningssagkyndige side 10,Håndbog for beskikkede bygningssagkyndige \n1...,1043
57,57,Håndbog for beskikkede bygningssagkyndige side 11,Håndbog for beskikkede bygningssagkyndige \n1...,796
58,58,Håndbog for beskikkede bygningssagkyndige side 12,Håndbog for beskikkede bygningssagkyndige \n1...,984
59,59,Håndbog for beskikkede bygningssagkyndige side 13,Håndbog for beskikkede bygningssagkyndige \n1...,312
60,60,Håndbog for beskikkede bygningssagkyndige side 14,Håndbog for beskikkede bygningssagkyndige \n1...,1033
61,61,Håndbog for beskikkede bygningssagkyndige side 15,Håndbog for beskikkede bygningssagkyndige \n1...,330
62,62,Håndbog for beskikkede bygningssagkyndige side 16,Håndbog for beskikkede bygningssagkyndige \n1...,831
63,63,Håndbog for beskikkede bygningssagkyndige side 17,Håndbog for beskikkede bygningssagkyndige \n1...,1117
64,64,Håndbog for beskikkede bygningssagkyndige side 18,Håndbog for beskikkede bygningssagkyndige \n1...,1070
65,65,Håndbog for beskikkede bygningssagkyndige side 19,Håndbog for beskikkede bygningssagkyndige \n1...,391


In [21]:
prompt = "Hvad koster nye vinduer?"
response, sections_tokens = answer_with_gpt_4(prompt, df, document_embeddings)
print(response)

Selected 3 document sections:
87
18
16

Svaret er ikke i ERFA bladene, håndbogen eller Sikkerhedsstyrelsens guider.
